In [19]:
%pip install pyspark
%pip install fhir.resources

# Directory containing Synthea-generated JSON files
input_directory = "/home/jovyan/work/fhir-data/fhir"
output_directory = "/home/jovyan/work/fhir-data/fhir-parquet"



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [20]:
import os
import json
from typing import Dict, Any, List, Set
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType
from collections import Counter
import traceback

# Initialize a SparkSession
spark = SparkSession.builder.appName("FHIR to Parquet").getOrCreate()

# Configuration
INPUT_DIRECTORY = "/home/jovyan/work/fhir-data/fhir"
OUTPUT_DIRECTORY = "/home/jovyan/work/fhir-data/fhir-parquet"

def discover_resource_types(directory: str) -> Set[str]:
    """Discover all FHIR resource types in the input directory."""
    resource_types = set()
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".json"):
                file_path = os.path.join(root, filename)
                with open(file_path, 'r') as file:
                    data = json.load(file)
                    entries = data.get('entry', [])
                    for entry in entries:
                        resource_type = entry['resource']['resourceType']
                        resource_types.add(resource_type)
    return resource_types

def parse_fhir_resource(resource_data: Dict[str, Any]) -> Row:
    """Parse a FHIR resource into a PySpark Row."""
    # Convert the entire resource to a JSON string
    return Row(resource=json.dumps(resource_data))

def parse_fhir_bundle(file_path: str, resource_types: Set[str]) -> Dict[str, List[Row]]:
    """Parse a FHIR bundle file and return resources grouped by type."""
    with open(file_path, 'r') as file:
        data = json.load(file)
        entries = data.get('entry', [])
        
        resources = {resource_type.lower() + "s": [] for resource_type in resource_types}
        
        for entry in entries:
            resource = parse_fhir_resource(entry['resource'])
            resource_type = entry['resource']['resourceType'].lower() + "s"
            if resource_type in resources:
                resources[resource_type].append(resource)
        
        return resources

def process_fhir_files(resource_types: Set[str]) -> Dict[str, List[Row]]:
    """Process all FHIR files in the input directory."""
    all_resources = {resource_type.lower() + "s": [] for resource_type in resource_types}
    resource_counts = Counter()
    
    for root, _, files in os.walk(INPUT_DIRECTORY):
        for filename in files:
            if filename.endswith(".json"):
                file_path = os.path.join(root, filename)
                resources = parse_fhir_bundle(file_path, resource_types)
                for resource_type, rows in resources.items():
                    all_resources[resource_type].extend(rows)
                    resource_counts[resource_type] += len(rows)
    
    print("Resource counts before processing:")
    for resource_type, count in sorted(resource_counts.items()):
        print(f"{resource_type}: {count}")
    
    return all_resources

def save_to_parquet(resources: Dict[str, List[Row]]):
    """Save resources to Parquet files."""
    processed_counts = Counter()
    error_counts = Counter()
    
    # Define schema for all resource types
    schema = StructType([
        StructField("resource", StringType(), True)
    ])
    
    for resource_type, rows in sorted(resources.items()):
        if rows:
            print(f"\nProcessing {resource_type}...")
            print(f"Number of rows: {len(rows)}")
            
            try:
                df = spark.createDataFrame(rows, schema)
                print(f"DataFrame created successfully for {resource_type}")
                # df.printSchema()
                # df.show(5, truncate=False)
                
                output_path = os.path.join(OUTPUT_DIRECTORY, resource_type)
                df.write.mode('overwrite').parquet(output_path)
                print(f"Parquet file written successfully for {resource_type}")
                processed_counts[resource_type] = df.count()
            except Exception as e:
                print(f"Error processing {resource_type}: {e}")
                error_counts[resource_type] += 1
                print(traceback.format_exc())
    
    print("\nResource counts after processing:")
    for resource_type, count in sorted(processed_counts.items()):
        print(f"{resource_type}: {count}")
    
    print("\nResources that encountered errors:")
    for resource_type, count in sorted(error_counts.items()):
        print(f"{resource_type}: {count}")

# Main execution
discovered_resource_types = discover_resource_types(INPUT_DIRECTORY)
print("Discovered resource types:", ", ".join(sorted(discovered_resource_types)))

all_resources = process_fhir_files(discovered_resource_types)
save_to_parquet(all_resources)

# Don't stop the SparkSession here, as it might be needed for further operations in the notebook
# spark.stop()

Discovered resource types: Claim, Condition, DiagnosticReport, DocumentReference, Encounter, ExplanationOfBenefit, Immunization, Location, Organization, Patient, Practitioner, PractitionerRole, Provenance
Resource counts before processing:
claims: 344
conditions: 33
diagnosticreports: 344
documentreferences: 344
encounters: 344
explanationofbenefits: 344
immunizations: 472
locations: 60
organizations: 59
patients: 33
practitionerroles: 59
practitioners: 59
provenances: 33

Processing claims...
Number of rows: 344
DataFrame created successfully for claims
Parquet file written successfully for claims

Processing conditions...
Number of rows: 33
DataFrame created successfully for conditions
Parquet file written successfully for conditions

Processing diagnosticreports...
Number of rows: 344
DataFrame created successfully for diagnosticreports
Parquet file written successfully for diagnosticreports

Processing documentreferences...
Number of rows: 344
DataFrame created successfully for doc